In [1]:
import numpy as np 
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt

In [2]:
def load_spectra(path):
    for itr in range(7):
        spectra_iteration = pd.read_excel(path + str(itr) + '.xlsx').values
        wavelength = spectra_iteration[:,0]
        if itr == 0:
            spectra_all = spectra_iteration[:,1:]
        else:
            spectra_all = np.hstack((spectra_all, spectra_iteration[:,1:]))
    return spectra_all, wavelength 

In [3]:
AP_ROD, wavelength = load_spectra('../data_AP_ROD/')
AP_SPHERE, _ = load_spectra('../data_AP_SPHERE/')
SRVF_ROD, _ = load_spectra('../data_SRVF_ROD/')
SRVF_SPHERE, _ = load_spectra('../data_SRVF_SPHERE/')
euclidean_ROD, _ = load_spectra('../data_euclidean_ROD/')
euclidean_SPHERE, _ = load_spectra('../data_euclidean_SPHERE/')
peakwv_ROD, _ = load_spectra('../data_peakwv_ROD/')
peakwv_SPHERE, _ = load_spectra('../data_peakwv_SPHERE/')

### All the data from the sphere and rod optimizations in one array

In [4]:
SPHERES = np.hstack((peakwv_SPHERE, euclidean_SPHERE, SRVF_SPHERE, AP_SPHERE))
RODS = np.hstack((peakwv_ROD, euclidean_ROD, SRVF_ROD, AP_ROD))

In [5]:
target = pd.read_excel('Target_Rod.xlsx')
target = np.asarray(target)
xt = target[:,0]
rod_target = target[:,1]
target = pd.read_excel('Target_Sphere.xlsx')
target = np.asarray(target)
xt = target[:,0]
sphere_target = target[:,1]

In [6]:
def Euclidean(f1,f2):
    d = distance.euclidean(f1,f2)    
    return -d

In [7]:
Euclidean_dist_SPHERE = {}
dist = []
for i in range(SPHERES.shape[1]):
    d = Euclidean(sphere_target, SPHERES[:,i])
    dist.append(d)